In [ ]:
!pip install -q -U google-generativeai
!pip install crewai crewai-tools
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install google-cloud-api-keys


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.4 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from google.colab import userdata
from crewai import Agent, LLM, Task, Crew, Process

from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from crewai_tools import SerperDevTool


In [ ]:
from google.cloud import api_keys_v2
from google.cloud.api_keys_v2 import Key


def create_api_key(project_id: str, suffix: str) -> Key:
    """
    Creates and restrict an API key. Add the suffix for uniqueness.

    TODO(Developer):
    1. Before running this sample,
      set up ADC as described in https://cloud.google.com/docs/authentication/external/set-up-adc
    2. Make sure you have the necessary permission to create API keys.

    Args:
        project_id: Google Cloud project id.

    Returns:
        response: Returns the created API Key.
    """
    # Create the API Keys client.
    client = api_keys_v2.ApiKeysClient()

    key = api_keys_v2.Key()
    key.display_name = f"My first API key - {suffix}"

    # Initialize request and set arguments.
    request = api_keys_v2.CreateKeyRequest()
    request.parent = f"projects/{project_id}/locations/global"
    request.key = key

    # Make the request and wait for the operation to complete.
    response = client.create_key(request=request).result()

    print(f"Successfully created an API key: {response.name}")
    # For authenticating with the API key, use the value in "response.key_string".
    # To restrict the usage of this API key, use the value in "response.name".
    print(response)
    return response


In [ ]:
import os
os.environ["SERPER_API_KEY"] = "a5be02241a04d38b452f2ae027aabcd73306019f"


In [ ]:
print(os.getenv("SERPER_API_KEY"))


a5be02241a04d38b452f2ae027aabcd73306019f


In [ ]:
tool = SerperDevTool()
# call gemini model
llm = LLM(
    api_key=os.getenv("GOOGLE_API_KEY"),
    model="gemini/gemini-1.5-flash",
)

In [ ]:
# Trip Researcher Agent
trip_researcher = Agent(
    role="Trip Researcher",
    goal="Research the destination: {destination}, and provide detailed information about attractions, culture, and travel tips.",
    backstory="You are a seasoned traveler with extensive knowledge of destinations worldwide. You specialize in uncovering hidden gems and practical travel advice.",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

# Trip Planner Agent
trip_planner = Agent(
    role="Trip Planner",
    goal="Create a multi-day travel itinerary for {destination} based on research, weather, and user preferences.",
    backstory="You are an expert in travel planning, crafting detailed itineraries tailored to individual preferences, interests, and conditions.",
    verbose=True,
    llm=llm
)

In [ ]:
# Research Task
research = Task(
    description=(
        "1. Gather information about the destination, including top attractions, cultural insights, local food, and travel tips.\n"
        "2. Investigate interesting activities and experiences unique to the destination.\n"
        "3. Provide a summary of findings with recommendations for must-visit places and must-try experiences."
    ),
    expected_output="A detailed report on the destination, including attractions, culture, food, and travel tips.",
    agent=trip_researcher,
)

# Planning Task
plan = Task(
    description=(
        "1. Develop a multi-day travel itinerary for the destination, considering the research findings, weather conditions, and user preferences.\n"
        "2. Include daily plans with activities, dining suggestions, and logistics tips.\n"
        "3. Adjust recommendations based on weather and preferences (e.g., mountains, beaches, cultural sites).\n"
        "4. Tailor the itinerary to the type of vacation (e.g., adventure, relaxation, cultural)."
    ),
    expected_output="A detailed multi-day travel itinerary with daily activities, dining suggestions, and travel tips, adjusted for weather and user preferences.",
    output_file="itinerary.md",
    agent=trip_planner,
)

In [ ]:
# Crew Definition
crew = Crew(
    agents=[trip_researcher, trip_planner],
    tasks=[research, plan],
    verbose=True
)

# User Input for Destination and Preferences
destination = input("Enter the destination you'd like to travel to: ")
weather = input("Enter the weather condition (e.g., sunny, cold, snowy): ").lower()
place_type = input("Enter your place preference (e.g., mountain, beach, city): ").lower()
vacation_type = input("Enter the type of vacation (e.g., adventure, relaxation, cultural): ").lower()

# Display the User's Choices
print(f"\nYou chose the following:\nDestination: {destination}\nWeather: {weather}\nPlace Preference: {place_type}\nVacation Type: {vacation_type}\n")

# Kickoff Process
result = crew.kickoff(inputs={
    'destination': destination,
    'weather': weather,
    'place_type': place_type,
    'vacation_type': vacation_type
})
print(result)

Enter the destination you'd like to travel to: Stockton
Enter the weather condition (e.g., sunny, cold, snowy): windy
Enter your place preference (e.g., mountain, beach, city): city
Enter the type of vacation (e.g., adventure, relaxation, cultural): relaxation

You chose the following:
Destination: Stockton
Weather: windy
Place Preference: city
Vacation Type: relaxation

# Agent: Trip Researcher
## Task: 1. Gather information about the destination, including top attractions, cultural insights, local food, and travel tips.
2. Investigate interesting activities and experiences unique to the destination.
3. Provide a summary of findings with recommendations for must-visit places and must-try experiences.
 

I encountered an error while trying to use the tool. This was the error: unhashable type: 'dict'.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'des